# IANNwTF HW 4
## Group 10

The following contains our solution to the exercises in IANNwTF HW 04. A Jupyter notebook versus a module format was chosen this time for purposes of organization.

### Assigment 1: Reviews
We review the homeworks for Groups 15 and 32.

### Assignment 2: MNIST Math

### 2.1 Preparing the MNIST Math Dataset

In [5]:
# Needed Imports
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

In [ ]:
# 2.1 Load Dataset
(train_ds, test_ds), ds_info = tfds.load ('mnist', split =['train', 'test'], as_supervised = True, with_info = True)

# Info on the Dataset (refresher)
print("ds_info: \n", ds_info)
tfds.show_examples(train_ds, ds_info)

In [7]:
# 2.2 Data Pipeline
def prepare_data(dataset, batchsize):

    '''
    :param dataset: the dataset to be prepared for input into the network
    :return: 2 datasets, one each for each of the math problems defined (see below), created after the original database was preprocessed with the
    steps below
    '''

    # Step One - General Preprocessing

    # convert data from uint8 to float32
    dataset = dataset.map(lambda img, target: (tf.cast(img, tf.float32), target))

    # flatten the images into vectors
    dataset = dataset.map(lambda img, target: (tf.reshape(img, (-1,)), target))

    # input normalization, just bringing image values from range [0, 255] to [-1, 1]
    dataset = dataset.map(lambda img, target: ((img / 128.) - 1., target))

    # Step 2 - Pairing Data Tuples & Respective Parameterized Targets

    # create a dataset that contains 2000 samples from the overall dataset paired with 2000 other samples
    data = tf.data.Dataset.zip((dataset.shuffle(2000),dataset.shuffle(2000)))

    # create the dataset for the first math problem (a + b >= 5) - remembering to cast to int versus boolean!
    greateqfive = data.map(lambda x1, x2: (x1[0], x2[0], x1[1]+x2[1]>=5))
    greateqfive = greateqfive.map(lambda x1, x2, t: (x1, x2, tf.cast(t, tf.int32)))

    # create the dataset for the second math problem (a - b = y)
    subtr = data.map(lambda x1, x2: (x1[0], x2[0], x1[1]-x2[1]))

    # Step 3 - Batching & Prefetching

    # run batching and prefetching for both datasets
    greateqfive = greateqfive.batch(batchsize)
    greateqfive = greateqfive.prefetch(tf.data.AUTOTUNE)
    subtr = subtr.batch(batchsize)
    subtr = subtr.prefetch(tf.data.AUTOTUNE)

    # return BOTH datasets
    return greateqfive, subtr

In [ ]:
# Check data pipeline by examining one example from each of the four created datasets (one for each math problem for train and test)

train_ds_gef, train_ds_subtr = prepare_data(train_ds, batchsize = 32)
test_ds_gef, test_ds_subtr = prepare_data(test_ds, batchsize = 32)

for img1,img2,label in train_ds_gef.take(1):
    print(img1.shape,img2.shape,label.shape)

for img1,img2,label in train_ds_subtr.take(1):
    print(img1.shape,img2.shape,label.shape)

for img1,img2,label in test_ds_gef.take(1):
    print(img1.shape,img2.shape,label.shape)

for img1,img2,label in test_ds_subtr.take(1):
    print(img1.shape,img2.shape,label.shape)


### Assignment 3: Building Shared Weight Models

In [9]:
class MyModel(tf.keras.Model):
    def __init__(self, activation, outputunits, optimizer):

        '''
        creates a neural network model with 2 hidden layers and an output layer
        '''

        super(MyModel, self).__init__()

        # create 2 hidden layers with 256 units and ReLU as the activation function
        self.hidden_layer_1 = Dense(units=256, activation=tf.nn.relu)
        self.hidden_layer_2 = Dense(units=256, activation=tf.nn.relu)

        # Concatenation Layer?

        # create an output layer with the specified number of output units and the chosen activation function
        self.output = Dense(units=outputunits, activation=activation)


    @tf.function
    def __call__(self, input1, input2):

        '''
        :param input: input to the network (tensor)
        :return: output of final layer
        '''

        i1 = self.hidden_layer_1(input1)
        i1 = self.hidden_layer_2(i1)
        i1 = self.output_layer(i1)

        i2 = self.hidden_layer_1(input2)
        i2 = self.hidden_layer_2(i2)
        i2 = self.output_layer(i2)

        # Concatenation Layer?

        return ?

### Assignment 4: Training the Networks

In [12]:
def training(subtask,optimizer):
    '''
    :param subtask: defines the subtask to be solved, 0 is a + b >= 5, 1 is a - b = y
    :param optimizer: the optimizer function to use (based on the task)
    :return:
    '''

    # Create the model based on the settings above

    #Note - ignore the fact that train_ds and test_ds may be flagged as not defined; when the whole program is run, this should not be an issue
    train_ds_gef, train_ds_subtr = prepare_data(train_ds, batchsize = 32)
    test_ds_gef, test_ds_subtr = prepare_data(test_ds, batchsize = 32)

    if subtask == 0:
        activation = tf.nn.softmax
        outputunits = 10
        train_ds = train_ds_gef
        test_ds = test_ds_gef

    else:
        activation = tf.nn.sigmoid
        outputunits = 2
        train_ds = train_ds_subtr
        test_ds = test_ds_subtr

    # Initiate a model with the requested parameters

    network = MyModel(activation,outputunits,optimizer)

    # Train the model

In [ ]:
# Train a model to solve the first math problem
# Binary Crossentropy is good for binary classification problems
training(0,tf.keras.losses.BinaryCrossentropy)

In [ ]:
# Train a model to solve the second math problem
# Mean Squared Error is good for continuous output problems
training(1,tf.keras.losses.MeanSquaredError)

### Assignment 5 - Experiments

Run training w/ classic SGD (no momentum)

Run training w/ Adam

Run training w/ SGD + Momentum

Run training w/ RMSrop

Run training w/ AdaGrad

In [ ]:
# Visualize the results of the above training runs

